In [1]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Create directory if it doesn't exist
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Copy kaggle.json file
os.system("cp kaggle.json ~/.kaggle/")

# Change permissions of kaggle.json file
os.system("chmod 600 ~/.kaggle/kaggle.json")


1

In [3]:
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
from zipfile import ZipFile
senti= 'C:\\Users\\HP\\sentiment140.zip'

with ZipFile(senti,'r') as file:
    file.extractall()
    print('Extracted')

Extracted


In [5]:
pip install --upgrade scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
column=['target','id','date','flag','user','text']

In [10]:

df=pd.read_csv('C:\\Users\\HP\\training.csv',names=column,encoding='latin-1')

In [11]:
df.head(10)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [12]:
df.shape

(1600000, 6)

In [13]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [14]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [15]:
df.replace({'target':{4:1}}, inplace=True)

In [16]:
df['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [17]:
st=PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]', ' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[st.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    
    return stemmed_content


In [19]:
df['stemmed_content']=df['text'].apply(stemming)

In [20]:
df.head(5)

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [27]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [28]:
x=df['stemmed_content'].values
y=df['target'].values

In [29]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [30]:
print(y)

[0 0 0 ... 1 1 1]


In [115]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.3,stratify=y,random_state=2)

In [116]:
print(x.shape,X_train.shape,X_test.shape)

(1600000,) (1120000,) (480000,)


In [117]:
print(y.shape,Y_train.shape,Y_test.shape)

(1600000,) (1120000,) (480000,)


In [118]:
X_train

array(['finish read book second time',
       'tommygun back action lunch today best watchout bday boy take',
       'kata shall listen later', ..., 'jedbramwel thank teddytuesday',
       'mtv movi award hope rob win realli care twilight win best movi rob win',
       'love wake folger bad voic deeper'], dtype=object)

In [119]:
X_test

array(['gylesoneshow sure find think may dead sorri break way',
       'final updat turn taken hr', 'rain', ...,
       'soon go sligo coupl day bankholiday tomorrow see ya',
       'piaaguirr know right grabe manira', 'feelin empti lt'],
      dtype=object)

In [120]:
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [121]:
print(X_train)

  (0, 374778)	0.31949818170660904
  (0, 328004)	0.5314491135690684
  (0, 43969)	0.4748653389926461
  (0, 306319)	0.4371164314704369
  (0, 121348)	0.44599875194306554
  (1, 360908)	0.20232270128356875
  (1, 45367)	0.24454930105105102
  (1, 32222)	0.2885646935506797
  (1, 400139)	0.5008486447803013
  (1, 36162)	0.21760975021723114
  (1, 376966)	0.16230856596272633
  (1, 224362)	0.24716194206021685
  (1, 2941)	0.3329517305729671
  (1, 28528)	0.1699742942038265
  (1, 377777)	0.5390797727604116
  (2, 209149)	0.36506885244060977
  (2, 217307)	0.35862123416780045
  (2, 331146)	0.45772430309259293
  (2, 192775)	0.7270515839609045
  (3, 54144)	0.3405985100366321
  (3, 381278)	0.3044716859774938
  (3, 114004)	0.3165169852577187
  (3, 344001)	0.3565976148233519
  (3, 366377)	0.4018889828996008
  (3, 139076)	0.25759117714273316
  :	:
  (1119996, 327788)	0.2626904633863284
  (1119996, 133766)	0.14165568151908714
  (1119996, 400064)	0.17675763929108915
  (1119996, 366962)	0.1645582384006955
  (11199

In [122]:
print(X_test)

  (0, 400455)	0.24464092290138587
  (0, 372248)	0.2111379764309065
  (0, 357350)	0.2678444483614686
  (0, 345122)	0.24201085278417414
  (0, 235076)	0.29762268984383516
  (0, 143634)	0.6303950980554028
  (0, 121198)	0.2668513552052889
  (0, 86927)	0.33413484518869224
  (0, 46696)	0.3105296872456904
  (1, 390597)	0.41919925544830766
  (1, 383626)	0.43305234092570793
  (1, 360939)	0.5151309759331816
  (1, 156589)	0.48451365961749343
  (1, 121102)	0.36962717098251946
  (2, 303665)	1.0
  (3, 281474)	0.6528792362184421
  (3, 156926)	0.2493726437719803
  (3, 72122)	0.26777644261468114
  (3, 39958)	0.6085945392762492
  (3, 28528)	0.2635724776921008
  (4, 345184)	0.51446657839167
  (4, 220487)	0.25816571428786494
  (4, 220281)	0.3037998272593176
  (4, 215156)	0.21842711142276236
  (4, 189400)	0.463921117899177
  :	:
  (479996, 229032)	0.2679813093524318
  (479996, 221882)	0.13337348202484534
  (479996, 146137)	0.16911987546565427
  (479996, 118268)	0.25120743997882045
  (479996, 112760)	0.24731

In [123]:
model=LogisticRegression(max_iter=10000)

In [124]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=10000)

In [125]:
predx=model.predict(X_train)
acc=accuracy_score(Y_train,predx)

In [126]:
acc

0.8039133928571428

In [127]:
predc=model.predict(X_test)
acc1=accuracy_score(Y_test,predc)

In [128]:
acc1

0.77708125

In [130]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [132]:
#Loading the saved model

In [131]:
loaded_model=pickle.load(open('C:\\Users\\HP\\trained_model.sav','rb'))

In [139]:
x_new=X_test[30000]
print(Y_test[30000])

prediction=loaded_model.predict(x_new)
print(prediction)

if (prediction[0]==0):
    print('Negative Tweet')
    
else :
    print('Positive Tweet')

0
[0]
Negative Tweet
